In [1]:
#### DO NOT RUN THIS CODE IN THE NOTEBOOK - copy paste into terminal in the HOME directory!

# git clone https://github.com/tensorflow/models.git

In [2]:
# !pip install s3fs -q
# !pip install tensorflow_hub -q
# !pip install tensorflow_datasets -q
# !pip install pycocotools -q
# !pip install gin-config -q
# !pip install immutabledict -q
# !pip install sentencepiece -q
# !pip install transformers -q
# !pip install evaluate -q
# !pip install sacrebleu -q
# !pip install --upgrade keras-nlp -q
# !pip install --upgrade tensorflow-model-optimization -q
# !pip install pyarrow fastparquet -q
# !pip install -r official/projects/movinet/requirements.txt -q

In [3]:
import configparser

config = configparser.ConfigParser()
config.read('/home/ec2-user/.aws/credentials')

['/home/ec2-user/.aws/credentials']

In [4]:
# AWS credentials and S3 settings
aws_access_key_id = config["default"]['aws_access_key_id']
aws_secret_access_key = config["default"]['aws_secret_access_key']
bucket_name = 'asl-capstone'
s3_URI = 's3://asl-capstone/'

In [5]:
import s3fs
fs = s3fs.S3FileSystem(key=aws_access_key_id, secret=aws_secret_access_key)

import boto3
s3 = boto3.client('s3',aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_access_key,
                  region_name = 'us-west-2')


In [6]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

# import evaluate
# import tensorflow_model_optimization as tfmot

from sklearn.model_selection import train_test_split

tf.keras.backend.clear_session()

2023-11-19 05:31:08.822055: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-19 05:31:09.528152: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
# !export XLA_PYTHON_CLIENT_PREALLOCATE=false
# !export XLA_FLAGS="--xla_gpu_strict_conv_algorithm_picker=false --xla_gpu_force_compilation_parallelism=1"

In [8]:
# Load model directly
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM, TFT5ForConditionalGeneration #, TFT5EncoderModel
# from transformers.modeling_tf_outputs import TFBaseModelOutputWithPastAndCrossAttentions

# with tf.device("/GPU:0"):
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-base")
    # encoder_model = T5EncoderModel.from_pretrained("google/t5-v1_1-large")

model.trainable = True

# tokenizer = AutoTokenizer.from_pretrained("t5-large")
# model = TFT5ForConditionalGeneration.from_pretrained("t5-large")
# encoder_model = T5EncoderModel.from_pretrained("t5-large")

ENCODER_DIM = 768
# ENCODER_DIM = 1024


model.summary()

/opt/tensorflow/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:158: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
2023-11-19 05:31:12.978676: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See 

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  24674304  
                                                                 
 encoder (TFT5MainLayer)     multiple                  109628544 
                                                                 
 decoder (TFT5MainLayer)     multiple                  137949312 
                                                                 
Total params: 222903552 (850.31 MB)
Trainable params: 222903552 (850.31 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
# model.trainable = True

# for i in range(16):
#     model.encoder.block[-(i+1)].trainable = False

# for i in range(20):
#     model.decoder.block[-(i+1)].trainable = False

In [10]:
video_files = pd.read_parquet("s3://asl-capstone/youtube-asl/1000-samples/numpy_files/masterfile_final.parquet")

# video_files = pd.read_parquet("s3://asl-capstone/youtube-asl/test_sample/numpy_files/RGB/masterfile.parquet")
video_files = video_files.drop_duplicates().reset_index(drop=True)
video_files['movinet_path'] = video_files.path.str.replace("/numpy_files/", '/movinet/backbone/')
# video_files['movinet_path'] = video_files.path.str.replace("/RGB/", '/movinet_output/')
video_files['caption_cleaned'] = video_files.caption.str.lower().str.replace('\n',' ').str.replace('  ',' ').str.replace('...','. ').str.replace('..','.')

print(video_files.shape)
video_files.head(10)

(59042, 4)


,caption,path,movinet_path,caption_cleaned
0,Hello everyone!,s3://asl-capstone/youtube-asl/1000-samples/num...,s3://asl-capstone/youtube-asl/1000-samples/mov...,hello everyone!
1,"I am Kelly Matthews,",s3://asl-capstone/youtube-asl/1000-samples/num...,s3://asl-capstone/youtube-asl/1000-samples/mov...,"i am kelly matthews,"
2,and I am the\nResearch Coordinator,s3://asl-capstone/youtube-asl/1000-samples/num...,s3://asl-capstone/youtube-asl/1000-samples/mov...,and i am the research coordinator
3,and Outreach Coordinator\nat NCDHR.,s3://asl-capstone/youtube-asl/1000-samples/num...,s3://asl-capstone/youtube-asl/1000-samples/mov...,and outreach coordinator at ncdhr.
4,My name is on that flyer,s3://asl-capstone/youtube-asl/1000-samples/num...,s3://asl-capstone/youtube-asl/1000-samples/mov...,my name is on that flyer
5,so maybe you have seen\nthat flyer posted,s3://asl-capstone/youtube-asl/1000-samples/num...,s3://asl-capstone/youtube-asl/1000-samples/mov...,so maybe you have seen that flyer posted
6,"in regard to DWW,\nDeaf Weight Wise Counselor.",s3://asl-capstone/youtube-asl/1000-samples/num...,s3://asl-capstone/youtube-asl/1000-samples/mov...,"in regard to dww, deaf weight wise counselor."
7,We are searching for potential\ncounselors to ...,s3://asl-capstone/youtube-asl/1000-samples/num...,s3://asl-capstone/youtube-asl/1000-samples/mov...,we are searching for potential counselors to j...
8,"Now, we will be hosting and\nproviding DWW ser...",s3://asl-capstone/youtube-asl/1000-samples/num...,s3://asl-capstone/youtube-asl/1000-samples/mov...,"now, we will be hosting and providing dww serv..."
9,through our partners\nin three locations:,s3://asl-capstone/youtube-asl/1000-samples/num...,s3://asl-capstone/youtube-asl/1000-samples/mov...,through our partners in three locations:


In [11]:
s3_files = fs.ls('s3://asl-capstone/youtube-asl/1000-samples/movinet/backbone/')
s3_files = ['s3://' + file for file in s3_files]
print(s3_files[0])
len(s3_files)

s3://asl-capstone/youtube-asl/1000-samples/movinet/backbone/--6bmFM9wT4_0.npy


2001

In [12]:
video_files = video_files[video_files['movinet_path'].isin(s3_files)]
video_files.head()

,caption,path,movinet_path,caption_cleaned
270,Sign with me!,s3://asl-capstone/youtube-asl/1000-samples/num...,s3://asl-capstone/youtube-asl/1000-samples/mov...,sign with me!
271,Number,s3://asl-capstone/youtube-asl/1000-samples/num...,s3://asl-capstone/youtube-asl/1000-samples/mov...,number
272,Number 1,s3://asl-capstone/youtube-asl/1000-samples/num...,s3://asl-capstone/youtube-asl/1000-samples/mov...,number 1
273,Number 2,s3://asl-capstone/youtube-asl/1000-samples/num...,s3://asl-capstone/youtube-asl/1000-samples/mov...,number 2
274,Number 3,s3://asl-capstone/youtube-asl/1000-samples/num...,s3://asl-capstone/youtube-asl/1000-samples/mov...,number 3


In [13]:
### Start at training mini model to convert from to correct shape for T5
# # Loss function: CosineSimilarity.
# # Minimize for cosine sim between avg label emb and avg vid emb (after flattening)?
# vid_labels = tokenizer(video_files['caption_cleaned'].to_list(), return_tensors="tf", padding=True)
# label_embeds = model.encoder.embed_tokens(vid_labels.input_ids)
# label_embeds_avg = tf.math.reduce_mean(label_embeds, axis=1)

In [14]:
train_files, test_files = train_test_split(video_files, train_size=0.7)
# test_files, val_files = train_test_split(test_files, train_size=0.7)

print(train_files.shape)
print(test_files.shape)
# print(val_files.shape)

(1400, 4)
(601, 4)


In [15]:
tf.random.set_seed(111)

# conv = tf.keras.layers.Conv2D(64, 2, strides=(2, 2))
# flatten = tf.keras.layers.Reshape((-1, ENCODER_DIM))

conv = tf.keras.layers.Reshape((-1, 8*8*168))
flatten = tf.keras.layers.Dense(ENCODER_DIM, activation='swish', name='vid_embedding')

In [16]:
def data_generator(file_paths_df, conv, flatten, n_frames=256):
    for _, row in file_paths_df.iterrows():
        with fs.open(row['movinet_path'], 'rb') as f:
            movinet_output = np.load(f)

        vid_embedding = conv(movinet_output)
        vid_embedding = flatten(vid_embedding)
        
        if vid_embedding.shape[1] < n_frames:
            # for shorter frame lengths, pad with zeros
            padding = tf.zeros((1, tf.constant(n_frames) - vid_embedding.shape[1], 
                                vid_embedding.shape[2]))
            
            attention_mask = tf.concat([tf.ones(vid_embedding.shape[1]), 
                                        tf.zeros(tf.constant(n_frames) - vid_embedding.shape[1])], 
                                       axis=0)
            
            vid_embedding = tf.concat([vid_embedding, padding], axis=1)
            
        elif vid_embedding.shape[1] > n_frames:
            # for longer frame lengths, interpolate
            vid_embedding = tf.image.resize(vid_embedding, 
                                            size=(vid_embedding.shape[0], n_frames), 
                                            method='nearest')
            attention_mask = tf.ones(vid_embedding.shape[1])
        
        else:
            # for exactly right size, just output attention_mask
            attention_mask = tf.ones(vid_embedding.shape[1])
        
        
        vid_embedding = tf.squeeze(vid_embedding, [0])
        label = tokenizer(row['caption_cleaned'], return_tensors="tf", 
                           padding='max_length', max_length=64).input_ids[0]
        # label = row['caption']
        
        yield {'inputs_embeds':vid_embedding, #'input_ids':None, 'training':True,
               'attention_mask':attention_mask, 'labels':label}


In [17]:
# TRY DATACOLLATOR or IterableDataset

# from datasets import IterableDataset
# train_dataset = IterableDataset.from_generator(lambda: data_generator(train_files[:32], conv, flatten, n_frames=256))

In [18]:
# list(train_dataset.take(1))

In [19]:
# training_args = transformers.Seq2SeqTrainingArguments()

In [20]:
# Create TensorFlow Dataset for model input. 
N_FRAMES = 512

train_dataset = tf.data.Dataset.from_generator(
    generator=lambda: data_generator(video_files, conv, flatten, n_frames=N_FRAMES),
    output_signature=({'inputs_embeds':tf.TensorSpec(shape=(N_FRAMES, ENCODER_DIM), dtype=tf.float32),
                       'attention_mask':tf.TensorSpec(shape=(N_FRAMES), dtype=tf.float32), 
                       'labels':tf.TensorSpec(shape=(64), dtype=tf.int32)}
                       ))

# test_dataset = tf.data.Dataset.from_generator(
#     generator=lambda: data_generator(test_files, conv, flatten, n_frames=N_FRAMES),
#     output_signature=({'inputs_embeds':tf.TensorSpec(shape=(N_FRAMES, ENCODER_DIM), dtype=tf.float32),
#                        'attention_mask':tf.TensorSpec(shape=(N_FRAMES), dtype=tf.float32), 
#                        'labels':tf.TensorSpec(shape=(64), dtype=tf.int32)}
#                        ))

# generation_dataset = tf.data.Dataset.from_generator(
#     generator=lambda: data_generator(val_files, conv, flatten, n_frames=N_FRAMES),
#     output_signature=({'inputs_embeds':tf.TensorSpec(shape=(N_FRAMES, ENCODER_DIM), dtype=tf.float32),
#                        'attention_mask':tf.TensorSpec(shape=(N_FRAMES), dtype=tf.float32), 
#                        'labels':tf.TensorSpec(shape=(64), dtype=tf.int32)}
#                        ))

In [21]:
# mt_metrics = evaluate.combine(["bleu", "chrf"], force_prefix=True)
# def compute_metrics(pred):
#     labels_ids = pred[0].label_ids
#     pred_ids = pred[0].predictions
    
#     predictions = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)

#     labels_ids[labels_ids == -100] = tokenizer.pad_token_id
#     references = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

#     outputs = mt_metrics.compute(predictions=predictions,
#                              references=references)

#     return outputs


# import keras_nlp

# rouge_l = keras_nlp.metrics.RougeL()


# def metric_fn(eval_predictions):
#     predictions, labels = eval_predictions
#     decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     for label in labels:
#         label[label < 0] = tokenizer.pad_token_id  # Replace masked label tokens
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
#     result = rouge_l(decoded_labels, decoded_predictions)
#     # We will print only the F1 score, you can use other aggregation metrics as well
#     result = {"RougeL": result["f1_score"]}

#     return result


# from transformers.keras_callbacks import KerasMetricCallback

# metric_callback = KerasMetricCallback(
#     metric_fn, eval_dataset=generation_dataset, predict_with_generate=True
# )

# callbacks = [metric_callback]

In [22]:
# # FIGURE OUT OPTIMIZER FOR t5

NUM_EPOCHS = 2
BATCH_SIZE = 16
VAL_BATCH_SIZE = 16

train_steps = len(train_files) // BATCH_SIZE
total_train_steps = train_steps * NUM_EPOCHS

test_steps = len(test_files) // VAL_BATCH_SIZE

# Set learning rate scheduler, optimizer, and loss function
initial_learning_rate = 1e-3
learning_rate = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate, decay_steps=total_train_steps,
)

optimizer = tf.keras.optimizers.Adafactor(learning_rate=learning_rate)

loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(
                from_logits=True)

model.compile(optimizer=optimizer, loss=model.hf_compute_loss) #loss=model.hf_compute_loss

In [29]:
# quantize_model = tfmot.quantization.keras.quantize_model

# # q_aware stands for for quantization aware.
# q_aware_model = quantize_model(model)

# q_aware_model.compile(optimizer=optimizer, loss=model.hf_compute_loss)

# q_aware_model.summary()

In [23]:
%%time

# with tf.device("/GPU:0"):
history = model.fit(train_dataset.batch(BATCH_SIZE,drop_remainder=True), 
                      # validation_data=test_dataset.batch(VAL_BATCH_SIZE,drop_remainder=True),
                      batch_size = BATCH_SIZE,
                      # validation_batch_size = VAL_BATCH_SIZE,
                      epochs=NUM_EPOCHS, 
                      # predict_with_generate = True,
                      # steps_per_epoch=train_steps,
                      # validation_steps=test_steps,
                      verbose=1)

Epoch 1/2


2023-11-19 05:32:49.006455: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f26a0128220 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-19 05:32:49.006491: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A10G, Compute Capability 8.6
2023-11-19 05:32:49.012261: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-19 05:32:49.296120: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8903
2023-11-19 05:32:49.478194: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


    125/Unknown - 1027s 8s/step - loss: 1.1309

2023-11-19 05:49:24.585466: I tensorflow/core/framework/local_rendezvous.cc:409] Local rendezvous send item cancelled. Key hash: 5522604598348518138
2023-11-19 05:49:24.585513: I tensorflow/core/framework/local_rendezvous.cc:409] Local rendezvous send item cancelled. Key hash: 8338759124600574948
2023-11-19 05:49:24.585518: I tensorflow/core/framework/local_rendezvous.cc:409] Local rendezvous send item cancelled. Key hash: 9831261058010758832
2023-11-19 05:49:24.585523: I tensorflow/core/framework/local_rendezvous.cc:409] Local rendezvous send item cancelled. Key hash: 9776252795544373010
2023-11-19 05:49:24.585527: I tensorflow/core/framework/local_rendezvous.cc:409] Local rendezvous send item cancelled. Key hash: 6174684428843410757
2023-11-19 05:49:24.585531: I tensorflow/core/framework/local_rendezvous.cc:409] Local rendezvous send item cancelled. Key hash: 7998399940226674279
2023-11-19 05:49:24.585535: I tensorflow/core/framework/local_rendezvous.cc:409] Local rendezvous send ite

125/125 [==============================] - 1027s 8s/step - loss: 1.1309
Epoch 2/2
125/125 [==============================] - 949s 8s/step - loss: 0.7816
CPU times: user 5min 27s, sys: 1min 5s, total: 6min 32s
Wall time: 32min 56s


In [25]:
test = next(iter(train_dataset.batch(8)))
test_outputs = model.generate(inputs_embeds = test['inputs_embeds'], attention_mask=test['attention_mask'])
predictions = tokenizer.batch_decode(test_outputs, skip_special_tokens=True, max_new_tokens=20, num_beams=2)
test_labels = tokenizer.batch_decode(test['labels'], skip_special_tokens=True)

for pred, lab in zip(predictions, test_labels):
    print('prediction: ', pred)
    print('true label: ', lab)
    print('')

/opt/tensorflow/lib/python3.10/site-packages/transformers/generation/tf_utils.py:838: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


prediction:  
true label:  sign with me!

prediction:  
true label:  number

prediction:  
true label:  number 1

prediction:  
true label:  number 2

prediction:  
true label:  number 3

prediction:  
true label:  number 4

prediction:  
true label:  number 5

prediction:  
true label:  number 6



In [ ]:
# # training
# inputs = tokenizer("The <extra_id_0> walks in <extra_id_1> park", return_tensors="tf").input_ids
# labels = tokenizer("<extra_id_0> cute dog <extra_id_1> the <extra_id_2>", return_tensors="tf").input_ids
# outputs = model(inputs, labels=labels)
# loss = outputs.loss
# logits = outputs.logits
# print(loss)
# print(logits)

# # inference
# inputs = tokenizer(
#    [ "studies have shown that owning a cat is good for you", "i have always wanted to have a pet"], return_tensors="pt", padding=True
# ).input_ids  # Batch size 1

# # Convert from tokens to embeddings
# encoder_outputs = encoder_model(input_ids=inputs)
# encoder_outputs['last_hidden_state'] = encoder_outputs['last_hidden_state'].detach().numpy()
# print(encoder_outputs['last_hidden_state'].shape)

# # Generate text from embeddings
# outputs = model.generate(encoder_outputs=encoder_outputs)
# # outputs = model.generate(inputs)


# print("Response: \n")
# print(tokenizer.batch_decode(outputs, skip_special_tokens=True))
# # studies have shown that owning a dog is good for you

In [81]:
# # training
# embeddings = tf.convert_to_tensor(vid_embedding)
# mask = tf.convert_to_tensor(attention_mask)

# labels = tokenizer(captions, return_tensors="tf", padding=True).input_ids
# outputs = model(input_ids=None, inputs_embeds=embeddings, 
#                      attention_mask=mask, labels=labels, training=True)
# print(outputs.loss)
# # loss = outputs.loss
# # logits = outputs.logits
# # print(loss)
# # print(logits)

2023-11-09 22:06:51.449044: W tensorflow/core/framework/op_kernel.cc:1816] INVALID_ARGUMENT: required broadcastable shapes


InvalidArgumentError: Exception encountered when calling layer 'SelfAttention' (type TFT5Attention).

{{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} required broadcastable shapes [Op:AddV2] name: 

Call arguments received by layer 'SelfAttention' (type TFT5Attention):
  • hidden_states=tf.Tensor(shape=(1, 256, 1024), dtype=float32)
  • mask=tf.Tensor(shape=(1, 1, 256, 1024), dtype=float32)
  • key_value_states=None
  • position_bias=None
  • past_key_value=None
  • layer_head_mask=None
  • query_length=None
  • use_cache=False
  • training=True
  • output_attentions=False

In [21]:
# model.generate(encoder_outputs=encoder_outputs)
# tokenizer.decode(model.generate(inputs_embeds=encoder_outputs)[0], skip_special_tokens=True)

/opt/tensorflow/lib/python3.10/site-packages/transformers/generation/tf_utils.py:838: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
2023-11-08 04:39:47.301175: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55703850 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-08 04:39:47.301212: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A10G, Compute Capability 8.6
2023-11-08 04:39:47.408791: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-08 04:39:47.854060: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8903
2023-11-08 04:39:48.206243: I ./tensorflow/compiler/jit/device_compiler.h:186]

's its its it so it. so when it so it so it so it'

In [ ]:
# def movinet_T5_model(video_base_model, language_base_model,
#                       max_sequence_length=120,
#                       hidden_size = 1024,
#                       dropout=0.1,
#                       learning_rate=0.00005):
#     """
#     Builds a translation model that accepts videos, creates video embeddings using a MoViNet model,
#     and then passes the video embeddings into an encoder-decoder to generate text translation
#     """

#     # Set all models to trainable
#     video_base_model.trainable = True
#     language_base_model.trainable = True

#     # Set input structure for video inputs
#     video_inputs = tf.keras.layers.Input(shape=(max_sequence_length, 256, 256, 3), dtype=tf.int64, name='input_layer')
#     # token_type_ids = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='token_type_ids_layer')
#     attention_mask = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='attention_mask_layer')

#     # Get output embeddings from video model
#     movinet_out = video_base_model(video_inputs)
#     vid_embedding = movinet_out[0]['block4_layer3']

#     # Layer to flatten w x h x c dimensions
#     vid_embedding_flatten = tf.keras.layers.Reshape((120, 8 * 8 * 168), name='flatten')(vid_embedding)

#     # Hidden layer to force embeddings into correct shape for language model
#     vid_embedding_flatten = tf.keras.layers.Dense(1024, activation='linear', name='hidden_layer')(vid_embedding_flatten)

#     # # Pass embeddings into T5 language model
#     # language_output = language_base_model.generate(encoder_outputs=encoder_outputs, attention_mask=attention_mask)

#     # classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(hidden)

#     # classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])

#     # classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
#     #                              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
#     #                              metrics='accuracy')



#     ### END YOUR CODE

#     model = tf.keras.Model(inputs=inputs, outputs=vid_embedding_flatten)

#     return model